In [2]:
import re
import gzip
import shutil
import pandas as pd
import sys
import io
import json
import re
import os
from pathlib import Path
import glob
from collections import defaultdict

In [20]:
python_files = glob.glob("pretrained_data/*.json")

In [21]:
print(python_files)

['pretrained_data\\python-000000000000.json']


### 1 Read All Pretrained Files

In [22]:
data = []
for file in python_files:
    with open(file, encoding="utf8") as f:
        entirefile = f.readlines()         
        for file in entirefile:
            file = eval(file)
            data.append(file)

### 2 Method Filtering

##### Repeated File

In [24]:
# remove repeated
content_dict = defaultdict(list)
for idx,d in enumerate(data):
    if "content" in d:
        content = d["content"]
        content_dict[content].append(idx)

In [25]:
print("Original file number: {}, unique file number: {}".format(len(data),len(content_dict)))

Original file number: 142982, unique file number: 111687


In [29]:
same_content_count = 0
filtered_data1 = []
for i in content_dict:
    if len(content_dict[i]) > 1:
        same_content_count += 1
    try:
        filtered_data1.append(content_dict[i][0])
    except:
        print(i)

##### Short File

In [30]:
number_lines = 20
filtered_data2 = []
for i in filtered_data1:
    if data[i]["content"].count("\n") > number_lines:
        filtered_data2.append(i)

In [31]:
print("Unique and long file number: {}".format(len(filtered_data2)))

Unique and long file number: 94448


##### Remove License

In [33]:
filtered_data3 = []
for i in filtered_data2:
    if "LICENSE" not in data[i]["path"]:
        filtered_data3.append(i)

In [34]:
print("Unique and long file and not license number: {}".format(len(filtered_data3)))

Unique and long file and not license number: 94448


##### Remove File that Contains Unpopular Libraries

### Store Data

In [35]:
processed_data = []
for idx in filtered_data3:
    processed_data.append(data[idx])

In [38]:
with open('processed_data/processed_pretrained_data.json', 'w') as f:
    json.dump(processed_data, f)

In [5]:
data2 = []
with open("processed_data/processed_pretrained_data.json") as f:
    data2 = json.load(f)

In [10]:
print(data2[0]["content"])

#!/usr/bin/env python

# Perform static code analysis

from __future__ import print_function

import os
import sys
import shutil
import re

# Severity levels
SEV_OK = 0
SEV_INFO = 1
SEV_WARNING = 2
SEV_ERROR = 3

SEV_NAMES = ['OK', 'Note', 'Warning', 'Error'];

# Utility functions

def init_totals():
	return [0 for i in range(SEV_ERROR + 1)];

def init_errors():
	return [[] for i in range(SEV_ERROR + 1)];
	
def find_scad_file(assembly):
    for filename in os.listdir(source_dir):
        if filename[-5:] == ".scad":
            #
            # look for module which makes the assembly
            #
            for line in open(source_dir + "/" + filename, "r").readlines():
                words = line.split()
                if len(words) and words[0] == "module":
                    module = words[1].split('(')[0]
                    if module == assembly:
                        return filename


    return None

# Faster regex
_regexp_compile_cache = {}

def recache(pattern):
    if 